## Evaluating potential overfitting in partitioning methods for phylogenetic inference

Even if information criteria like AIC, AICc, or BIC show better models with more partitions, the trees do not seem to be improving. Is this some sort of overfitting/overparameterizing?

Aim is to test this using simulated trees


From Jeremy's paper, the pipeline was:

Each concatenated alignment was partitioned using PartitionFinder v2 (Lanfear et al. 2017) and MtPAN(3) (Nardi et al. 2014) using the Gap_script_charset.pl, Gap_script_physio.pl, and Gap_script.R scripts (Nardi et al. 2014). Moreover, the dataset was partitioned by its original OGs. For MtPAN, we used k=10 clusters, using 10,000 k-means seeds and 250 simulations. The initial data blocks fed to PartitionFinder were the genes of each of the datasets. We used a relaxed clustering search approach (Lanfear et al. 2014), the linked branch lengths option, and all protein models of evolution were considered, with BIC model selection.

Partitioning schemes were inferred using RAxML and IQ-TREE. In RAxML (Stamatakis 2014), the tree search comprises of two main phases. During the first phase, the best candidate trees in terms of likelihood scores are re-evaluated, with the 20 best scoring trees saved. Beginning with an initial parsimony tree, the algorithm performs cycles of Subtree Pruning and Regrafting (SPR) moves, followed by branch length optimisations of the 20 best trees found during the SPRs. Here, each SPR cycle performs an SPR move on each of the subtrees and reinserts subtrees between a pre-determined rearrangement radius. If an improvement in likelihood is found, the tree is retained immediately. A more thorough rearrangement algorithm is then performed in order to find more likelihood improvements on the 20 best trees, in which branch lengths are optimised. In the second phase, candidate trees are evaluated more thoroughly, by altering the SPR radius. First, a smaller radius is used to apply SPRs, with the 20 best trees saved. If none of these 20 trees improve the tree, the radius in which SPRs are performed is extended, and so forth. IQ-TREE (Nguyen et al. 2015) uses a stochastic algorithm in order to find ML trees. It begins by generating 100 parsimony trees. From these, the 20 trees with the best ML score, and each with a unique topology, are selected. A stochastic hill climbing Nearest Neighbor Interchanges (NNI) algorithm is then used to find the best ML scoring trees. These trees are then further optimised in order to find the highest likelihood tree.

RAxML_Default trees were inferred using automatic model selection and a gamma model for rate heterogeneity, using the PROTGAMMAAUTO flag. The BIC criterion was used for model selection. Rapid bootstrap was implemented, with 100 replicates.

Similarly, for IQTREE_Default, the default settings, which implements ModelFinder (Kalyaanamoorthy et al. 2017) was implemented with a gamma model for rate heterogeneity. Ultrafast bootstraps was also implemented, with 1000 replicates (Hoang et al. 2018). Additionally, we inferred trees using ModelFinder without the constraint of a gamma model (IQTREE_FREERATE). These trees are included in the Supplementary materials.

For RAxML_Partition, IQTREE_Partition, RAxML_MtPAN and IQTREE_MtPAN, the ModelFinder implementation in IQTREE was used in order to find the most suitable model for each partition. For the IQTREE methods, all possible models of evolution were included, including mixture models. For the RAxML implementations, only models found in RAxML were included for selection (i.e., mixture models were not included). A gamma rate of heterogeneity was selected, as well as linked branch length models.

For both RAxML_PartitionFinder and IQTREE_PartitionFinder, a gamma rate of heterogeneity was used with the best scheme found by PartitionFinder. We also used edge-linked branch lengths when constructing phylogenetic trees. In the case of RAxML_PartitionFinder, 100 rapid bootstrap replicates were found. For IQTREE_PartitionFinder, 1000 ultrafast bootstrap replicates were inferred.

In addition to partitioning models, we also assess the performance of a protein mixture model. We used the GHOST model implemented in IQ-TREE, an edge unlinked mixture model designed to deal with heterotachy.

For our partitioned analysis, we used edge linked branch lengths. Edge unlinked branch lengths have 2n-3 extra parameters per additional partition (where n is the number of taxa), rather than a single extra parameter. Therefore, it is likely that edge unlinked models are over parameterised. The edge unlinked model was chosen for the IQTREE_GHOST method because it was used for its potential to account for heterotachy, which the edge unlinked model accounts for. For better comparison with the other methods in this study, the method should also be evaluated using the edge linked model.


#### First, simulate (a range of?) trees with ALF

In [ ]:
%%bash
alfsim inputfile

# or can do bash this way:
!alfsim inputfile

Then take the ALF output MSAs (mult. seq. align.s) and un-align them so that I can then try to build the trees from the unaligned sequences.

In [43]:
# cwd = os.getcwd()
# print(cwd)

import os # set the wd

from Bio import SeqIO, AlignIO
from zoo.wrappers.treebuilders import Phyml
from zoo.wrappers.treebuilders import Raxml
from zoo.wrappers.treebuilders import Iqtree
# regular expression needed to extract species name prior to concatenation
import re
# for IQtree to print a tree
import dendropy



# start with ALF output
alf_msa_dir = 'results/222dc49d-452f-43bf-a968-90e4fa814b6e/MSA/'

file = "MSA_1_aa.fa"

# eventually will loop through all sim outputs

msa = AlignIO.read(alf_msa_dir+"/"+file,format="fasta") 


print(msa)
        

SingleLetterAlphabet() alignment with 30 rows and 398 columns
AEE--TVLAV----------------------------------...KE- S001/00001
AKQ--TVIGL----------------------------------...SF- S002/00001
AQL--TGIDT----------------------------------...SW- S003/00001
ADD--TGIGS----------------------------------...RA- S004/00001
ASD--DGIRI----------------------------------...DI- S005/00001
AKS--TGIRP----------------------------------...RP- S006/00001
AV-----IRQ----------------------------------...E-- S007/00001
AGQ--TGIRI----------------------------------...RS- S008/00001
ADD--TGFRN----------------------------------...KN- S009/00001
KDADGSGVRE----------------------------------...RN- S010/00001
AKD--TGMGQ----------------------------------...SV- S011/00001
AQD--TGIRV----------------------------------...KT- S012/00001
ADT--TGLRW----------------------------------...AP- S013/00001
ADD--TSIRV----------------------------------...KN- S014/00001
ADD--TGFRI----------------------------------...KG- S015/00001
AEA--TGI

#### Partition the MSA from ALF

This is then fed into IQtree (don't think it will work for RaxML or Phyml)

Use a script from David to convert fasta format from ALF output to Phylip (relaxed) format for PartitionFinder2 input:

ACTUALLY, don't need to do this, at the bottom of the MSA directory for ALF output, it has one file with all alignments together in Phylip format. Otherwise if needed, David's script below does work, but throws an error when that .phy file is in the MSA directory.

In [48]:
# import glob, os
# from Bio import AlignIO

# input_folder = '/Users/kgilbert/Documents/UNIL/PartitioningMethods_SimulationProject/results/222dc49d-452f-43bf-a968-90e4fa814b6e/MSA/'

# for f in glob.glob(os.path.join(input_folder, '*.fa')):
#     input_handle = open(f, "r")
#     output_handle = open(f.replace('fa','phy'), "w")

#     align = AlignIO.read(input_handle, "fasta")
#     AlignIO.write(align, output_handle, "phylip-relaxed")

#     output_handle.close()
#     input_handle.close()

Put the cfg file and the input phylip file in the same folder...
Will need to iterate the input files that are listed in the first line of teh cfg file (cfg file is the parameter file for partitionfinder2).

In [73]:
# cannot run partitionfinder2 from here in jupyter. it uses python 2.7.10 or higher BUT not python 3.x
# was not able to install pytables dependency of partitionfinder2 with pip, 
#     had to resort to installing anaconda (v2.7)
# also could not manually instal pytables from github because latest version is too far ahead
#     to work with python2.7.10
# so for now, partitionfinder2 does work when run in the command line within the folder:
#       ~/Documents/UNIL//partition_finder_2/
# with the command:
#        python partitionfinder-2.1.1/PartitionFinderProtein.py partitionfinder-2.1.1/examples/aminoacid/
# sometimes the above command doesn't work, but then try again with a new terminal, 
#    or make a temporary anaconda environment as below, which can be deleted in /opt/anaconda2/envs/




## BELOW DOES NOT SEEM TO SOLVE THE ISSUE when trying to run partitionfinder2 in jupyter

#%%bash

#cd ../partition_finder_2/partitionfinder-2.1.1/

#/opt/anaconda2/bin/conda create -n test python

#/opt/anaconda2/bin/conda activate test

#python PartitionFinderProtein.py examples/aminoacid/

#/opt/anaconda2/bin/conda deactivate


Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/anaconda2/envs/test

  added / updated specs:
    - python


The following NEW packages will be INSTALLED:

  ca-certificates    pkgs/main/osx-64::ca-certificates-2019.10.16-0
  certifi            pkgs/main/osx-64::certifi-2019.9.11-py37_0
  libcxx             pkgs/main/osx-64::libcxx-4.0.1-hcfea43d_1
  libcxxabi          pkgs/main/osx-64::libcxxabi-4.0.1-hcfea43d_1
  libedit            pkgs/main/osx-64::libedit-3.1.20181209-hb402a30_0
  libffi             pkgs/main/osx-64::libffi-3.2.1-h475c297_4
  ncurses            pkgs/main/osx-64::ncurses-6.1-h0a44026_1
  openssl            pkgs/main/osx-64::openssl-1.1.1d-h1de35cc_3
  pip                pkgs/main/osx-64::pip-19.3.1-py37_0
  python             pkgs/main/osx-64::python-3.7.4-h359304d_1
  readline           pkgs/main/osx-64::readline-7.0-h1de35cc_5
  setuptools         pkgs/main/osx-64::setuptools-41.4.0-py37_0
  sqlite             pkgs/main/os

### Using the alignment, build the tree with PhyML

In [2]:
import sys

In [3]:
sys.executable

'/usr/local/bin/python3'

In [4]:
sys.path

['/Users/kgilbert/Documents/UNIL/PartitioningMethods_SimulationProject',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages',
 '/Users/kgilbert/Documents/UNIL/zoo',
 '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions',
 '/Users/kgilbert/.ipython']

In [5]:
!which jupyter

/Library/Frameworks/Python.framework/Versions/3.7/bin/jupyter


In [6]:
## PATH issues all fixed now, updated bash+profile and restarted jupyter

#!echo $PATH

#cwd = os.getcwd()
#print(cwd)


#!export PATH=$PATH:/Users/kgilbert/Documents/UNIL/phyml/src/
    
#!echo $PATH

#sys.path.append('/Users/kgilbert/Documents/UNIL/phyml/src/')

#!echo $PATH


In [7]:
tree = Phyml(msa, datatype="PROTEIN")#, binary='/Users/kgilbert/Documents/UNIL/phyml/src/')

In [8]:
phy_res = tree()

In [9]:
phy_res.print_plot()

/------------------------------------------------------------------- S004/00001
|                                                                              
|                                                            /------ S024/00001
|------------------------------------------------------------+                 
|                                                            \------ S021/00001
|                                                                              
|                                       /--------------------------- S019/00001
|                                       |                                      
|            /--------------------------+      /-------------------- S023/00001
|            |                          |      |                               
|            |                          \------+             /------ S027/00001
|            |                                 |      /------+                 
|            |                          

### Using the alignment, build the tree with RAxML

In [10]:
raxmltree = Raxml(msa, datatype="PROTEIN")

In [11]:
rax_res = raxmltree()

No match found for "alpha:" (at char 0), (line:1, col:1)


In [12]:
#rax_res.values()

rax_res['tree'].print_plot()

                                                              /----- S025/00001
     /--------------------------------------------------------+                
     |                                                        \----- S026/00001
     |                                                                         
     |     /-------------------------------------------------------- S023/00001
     |     |                                                                   
     |     |                                            /----------- S004/00001
     |     |                                       /----+                      
     |     |                                       |    |     /----- S021/00001
     |     |                                       |    \-----+                
     |     |          /----------------------------+          \----- S024/00001
/----+     |          |                            |                           
|    |     |          |                 

### Using the alignment, build the tree with IQTree

For input to IQtree, must have a partition file in RAxML or NEXUS format
for NEXUS, each partition could come from a separate alignment file; this is not possible for RAxML

In [13]:
iqtree = Iqtree(msa, datatype="PROTEIN")

In [15]:
iq_res = iqtree()

In [22]:
# old way before Adrian updated the wrapper:
# iqtree_res = dendropy.Tree.get(data=iq_res, schema='newick')
iq_res['tree'].print_plot()

/------------------------------------------------------------------- S001 00001
|                                                                              
|                                                             /----- S002 00001
|                                                  /----------+                
|                                                  |          \----- S011 00001
|                     /----------------------------+                           
|                     |                            |    /----------- S004 00001
|                     |                            \----+                      
|                     |                                 |     /----- S021 00001
|                     |                                 \-----+                
|                     |                                       \----- S024 00001
|                     |                                                        
|                     |                 

#### Now I can build all the trees, next need to use partitioning methods to see if/when we get better/worse trees depending on the AIC/BIC values

Lanfear et al 2008, MBE has PartitionFinder (v1 and v2)

You have to a priori define the number of data blocks? Which then creates the potential number of partitioning schemes...

#### IQTree uses partition methods during its tree building (different above?) so that would combine steps 2 and 3
still have to define where the partitions are a priori
3 types of partition models:
* edge-equal -- branch lengths same across all branches
* edge-proportional -- each partition has its own specific rate that rescales all of its branch lengths, i.e. diff. evolutionary rates between partitions
* edge-unlinked -- most parameter-rich model, each partition has its own set of branch lengths
they recommend edge-proportional, saying the 3rd might overfit and the 1st does not allow variation in speeds btwn partitions (so what does it allow????)


In [ ]:
## NOT USING ANY Of THIS CURRENTLY


#unalign_msa_dir = 'unaligned_MSA/'
# right now, 200 .fa files
#  infile = r"MSA_1_aa.fa"
#  outfile = r"unaligned_MSA_1_aa.fa"# un-align them (remove the "-" from all locations in all files)

# one way of doing it with a loop
#for i in range(1,5):  # change to 201 to do all 200 files
#    infile = "%sMSA_%d_aa.fa" % (alf_msa_dir, i)
#    #print(infile)
#    outfile = r"%sunaligned_MSA_%d_aa.fa" % (unalign_msa_dir, i)
#    
#    fin = open(infile,"r")
#    fout = open(outfile,"w+")
#    for line in fin:
#        line = line.replace("-", "")
#        fout.write(line)
#    fin.close()
#    fout.close()
#    print(i)

# fancier way of doing it, ala Christophe's code in treebuilding tutorial
# for file in os.listdir(alf_msa_dir):
#     if file.endswith(".fa"):
#         print(file)
#         infile = AlignIO.read(alf_msa_dir+'/'+file, "fasta")
#         outfile = unalign_msa_dir+'/'+file
#      
#         fin = open(infile,"r") ## THIS PART DOESN'T WORK, bc it is already in MSA format?
#         fout = open(outfile,"w+")
#         for line in fin:
#             line = line.replace("-", "")
#             fout.write(line)
#         fin.close()
#         fout.close()
        

## Do the inferred trees match the true tree?
### How to assess whether the trees are better recovered or not with partitioning/better AIC/BIC

- monophyly score - a measure of how well a phylogenetic tree recoveres monophyletic lineages
- Robinson Foulds distance - captures the variance in the best tree generated; looks at what inner groups are kept in some paritions?

Provide the Newick tree, and Jeremy had scripts that then calculate the two above measures. Needs a clade(???) file? I think this is just a list of all the samples, which for me would be the number of simulated species. Plus the Newick tree.

In [23]:
# Jeremy - Robinson-Foulds script
"""
Calculates RF distances between two trees
"""

from ete3 import Tree

def collapse_branches(TREE_FILE,SUPPORT_THRESHOLD):
    t = Tree(TREE_FILE)
    for node in t.get_descendants():
        if not node.is_leaf() and (node.support <= SUPPORT_THRESHOLD):
            node.delete()
    return t


def count_internal(tree):
    tree.unroot()
    edges=-1
    for edge in tree.traverse():
        if not edge.is_leaf():
            edges+=1
    return edges

def rf_distance(tree1,tree2,option=False):
#    if option=='collapse':
#        t1 = collapse_branches(tree1,0.75)
#        t2 = collapse_branches(tree2,0.75)
#        option = 'reduced'
#    else:
#        t1 = Tree(tree1)
#        t2 = Tree(tree2)

    t1.unroot()
    t2.unroot()

    rf = t1.robinson_foulds(t2,unrooted_trees=True)

    rf_dist = rf[0]
    max_rf = rf[1]
    num_leaves=len(rf[2])

    max_resolved_score = (2*num_leaves)-6

    internal_branches_t1 = count_internal(t1)
    internal_branches_t2 = count_internal(t2)

    total_internal = internal_branches_t1 + internal_branches_t2

    num_missing_splits_t1 = num_leaves - 3 - internal_branches_t1
    num_missing_splits_t2 = num_leaves - 3 - internal_branches_t2




    rf_dist_upper = rf_dist + num_missing_splits_t1 + num_missing_splits_t2


    #If normalise by (num intenral branches)
    if option=='reduced':
        normalised_rf = rf_dist/total_internal

    elif option=='upper':
    #If add score to create upper bound due to being polytomy
        normalised_rf = rf_dist_upper/max_resolved_score
    else:
        normalised_rf = rf_dist/max_resolved_score

    return normalised_rf


In [27]:
# let's compare these two trees
# later on --- use the actual tree from ALF sims...

#print(phy_res)

#print('\n')

#print(iqtree_res)

#print(rax_res['tree'])

## NEED TO APPEND A ';' TO THE TREE STRINGS, CAN'T SEEM TO DO THIS TO THE TREE FORMAT within python?
# write each tree to a file and then read from there for RF calcs

with open('iqtree.nw', 'w') as f:
    print(iq_res['tree'], ";", file=f)

with open('phyml.nw', 'w') as f:
    print(phy_res, ";", file=f)

with open('raxml.nw', 'w') as f:
    print(rax_res['tree'], ";", file=f)


In [28]:
t0 = Tree ("results/222dc49d-452f-43bf-a968-90e4fa814b6e/RealTree.nwk")

t1 = Tree("iqtree.nw")
t2 = Tree("phyml.nw")
t3 = Tree("raxml.nw")

t4=Tree("test_out.txt")

#print(t2)

Apply the RF script to my trees; compare original simulated tree to each inferred tree.

In [29]:
rf_distance(t0, t1) # vs iqtree

-0.0

In [30]:
rf_distance(t0, t2) # vs phyml

-0.0

In [31]:
rf_distance(t0, t3) # vs raxml

-0.0